<a href="https://colab.research.google.com/github/RaghunandanVenkatesh/LearningToSee/blob/master/RL_Hvac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline

In [2]:
#inputs for plant
T_oat = 30.0
PWM_front_box = 30.0
T_enginewater_set = 80.0
POS_fresh_air_flap = 40.0
value = 0
dt = 1 # sample time

In [3]:
# initialization 
water_val_pos_list = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
A_list = np.array([0, 0.6011, .61, 0.6, 1.88, 1.88, 2.1, 2.1, 2.1, 2.5, 3.5])
lookup = UnivariateSpline(water_val_pos_list, A_list, k=1, s=0.0)
T_enginewater = T_oat
T_fap = T_oat
hA_screen = 0.0007
hA_shell = 0.0029
convfactor = 0.0016
mcp_shell = 0.83
T_shell = T_oat
T_set = 24


# Engine water temp

In [4]:
T_enginewater = T_oat + ( T_enginewater_set + T_oat - T_enginewater) * (1 - np.exp(-value)) 
value = np.min([value+0.002, 5]) 

# Airoutlet temperature

In [5]:
T_air_in = T_oat * POS_fresh_air_flap/100 + T_fap * (1 - POS_fresh_air_flap/100)
A = lookup(POS_fresh_air_flap)
eff = 1 - np.exp(-POS_fresh_air_flap*A/100)
T_airout = T_air_in + (T_enginewater - T_air_in) * eff

# room temperature 

In [6]:
d_T_fap = hA_screen * (T_oat - T_fap) + hA_shell * (T_shell - T_fap)
d_T_fap += PWM_front_box * convfactor * 0.718 * (T_airout - T_fap)
T_fap = d_T_fap * dt + T_fap
T_shell = T_shell - hA_shell * (T_shell - T_fap)/mcp_shell

In [7]:
class Environment:
    def __init__(self, T_oat, PWM_front_box, T_enginewater_set, POS_fresh_air_flap, T_set ):
        self.T_oat = T_oat
        self.PWM_front_box = PWM_front_box
        self.T_enginewater_set = T_enginewater_set
        self.POS_fresh_air_flap = POS_fresh_air_flap
        self.value = 0
        self.dt = 1 # sample time
        self.T_set = T_set
        # initialization 
        water_val_pos_list = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
        A_list = np.array([0, 0.6011, .61, 0.6, 1.88, 1.88, 2.1, 2.1, 2.1, 2.5, 3.5])
        self.lookup = UnivariateSpline(water_val_pos_list, A_list, k=1, s=0.0)
        self.T_enginewater = T_oat
        self.T_fap = T_oat
        self.hA_screen = 0.0007
        self.hA_shell = 0.0029
        self.convfactor = 0.0016
        self.mcp_shell = 0.83
        self.T_shell = T_oat

    def step(self):
        # engine water temp
        self.T_enginewater = self.T_oat + ( self.T_enginewater_set + self.T_oat - self.T_enginewater) * (1 - np.exp(-self.value)) 
        self.value = np.min([self.value+0.002, 5])
        # air outlet temp
        T_air_in = self.T_oat * self.POS_fresh_air_flap/100 + self.T_fap * (1 - self.POS_fresh_air_flap/100)
        A = self.lookup(self.POS_fresh_air_flap)
        eff = 1 - np.exp(-self.POS_fresh_air_flap*A/100)
        T_airout = T_air_in + (self.T_enginewater - T_air_in) * eff 
        # room temperature
        d_T_fap = self.hA_screen * (self.T_oat - self.T_fap) + self.hA_shell * (self.T_shell - self.T_fap)
        d_T_fap += self.PWM_front_box * self.convfactor * 0.718 * (T_airout - self.T_fap)
        self.T_fap = d_T_fap * dt + self.T_fap
        self.T_shell = self.T_shell - self.hA_shell * (self.T_shell - self.T_fap)/self.mcp_shell    

        #reward
        reward = -(self.T_set - self.T_fap)**2
        observation = [self.T_fap]
        info = False
        return observation, reward, info

    def reset(self):
        self.T_shell = T_oat
        self.T_fap = T_oat
        self.T_enginewater = T_oat
        observation = [self.T_fap]
        return observation                     

In [8]:
upper_temp_fap = 60
min_temp_fap = -40
env = Environment( T_oat, PWM_front_box, T_enginewater_set, POS_fresh_air_flap, T_set )

In [9]:
env.step()

([30.0], -36.0, False)

In [10]:
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np

In [13]:
class HvacPlantEnv(gym.Env):
    def __init__(self, T_oat, T_enginewater_set, T_set):
        self.dt = 1 # sample time
        # initialization 
        water_val_pos_list = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
        A_list = np.array([0, 0.6011, .61, 0.6, 1.88, 1.88, 2.1, 2.1, 2.1, 2.5, 3.5])
        self.lookup = UnivariateSpline(water_val_pos_list, A_list, k=1, s=0.0)
        self.hA_screen = 0.0007
        self.hA_shell = 0.0029
        self.convfactor = 0.0016
        self.mcp_shell = 0.83
        self.T_oat = T_oat
        self.T_enginewater_set = T_enginewater_set
        self.T_set = T_set
        self.action_space = spaces.Tuple((
                                spaces.Discrete(100),
                                spaces.Discrete(100)))
        self.observation_space = spaces.Box(-40, 80, shape=(4,) ,dtype=np.float32)


        self.state = None

        self.steps_beyond_done = None

    def step(self, action):
        T_shell, T_fap, T_enginewater, value = self.state
        POS_fresh_air_flap = action[0]
        PWM_front_box = action[1] 
        # engine water temp
        T_enginewater = self.T_oat + ( self.T_enginewater_set + self.T_oat - T_enginewater) * (1 - np.exp(-value)) 
        value = np.min([value+0.002, 5])
        # air outlet temp
        T_air_in = self.T_oat * POS_fresh_air_flap/100 + T_fap * (1 - POS_fresh_air_flap/100)
        A = self.lookup(POS_fresh_air_flap)
        eff = 1 - np.exp(-POS_fresh_air_flap*A/100)
        T_airout = T_air_in + (T_enginewater - T_air_in) * eff 
        # room temperature
        d_T_fap = self.hA_screen * (self.T_oat - T_fap) + self.hA_shell * (T_shell - T_fap)
        d_T_fap += PWM_front_box * self.convfactor * 0.718 * (T_airout - T_fap)
        T_fap = d_T_fap * self.dt + T_fap
        T_shell = T_shell - self.hA_shell * (T_shell - T_fap)/self.mcp_shell 
        self.state = (T_shell, T_fap, T_enginewater, value)   

        #reward
        done  = self.T_set == T_fap
        if not done:
            reward = np.abs(self.T_set - T_fap)
        elif self.steps_beyond_done is  None:
            self.steps_beyond_done = 0
            reward = np.abs(self.T_set - T_fap)
        else:
            if self.steps_beyond_done == 0:
                print(
                  "You are calling 'step()' even though this "
                  "environment has already returned done = True. You "
                  "should always call 'reset()' once you receive 'done = "
                  "True' -- any further steps are undefined behavior.")
            self.steps_beyond_done += 1
            reward = 0.0
        return np.array(self.state), reward, done, {} 

    def reset(self):
        # self.T_oat = T_oat
        # self.T_enginewater_set = T_enginewater
        self.state = [self.T_oat, self.T_oat, self.T_oat, 0]
        self.steps_beyond_done = None
        return np.array(self.state)

In [14]:
action = spaces.Tuple((spaces.Discrete(10), spaces.Discrete(100))).sample()
env_h = HvacPlantEnv(24,80,30)
env_h.reset()
env_h.step(action)

(array([2.4e+01, 2.4e+01, 2.4e+01, 2.0e-03]), 6.0, False, {})

In [97]:
# env_h.action_space.sample()

(53, 60)

In [98]:
state = env_h.reset()
action = env_h.action_space.sample()
obs, rew , done, _ = env_h.step(action)
print(state.shape)
print(env_h.observation_space.shape)

(4,)
(4,)


In [99]:
# action_list = []
# for _ in range(50):
#     action = env_h.action_space.sample()
#     # print(action)
#     action_list.append(action)
# for i in range(len(action_list)):
#     obs,rew,done,_ = env_h.step(action_list[i])
#     print(obs)

In [100]:
from tqdm import tqdm

In [101]:
nb_steps =100
cum_rew = 0
i = 0
for i in tqdm(range(nb_steps)): 
    action = env_h.action_space.sample()
    x, reward, done, _ = env_h.step(action)
    #print(reward)
    cum_rew += rew
    i+=1
print(cum_rew/i)

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 14292.10it/s]

6.0


DDPG Agent

In [102]:
from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input, Concatenate
from tensorflow.keras import initializers, regularizers
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np

In [103]:
nb_actions = len(env_h.action_space.sample())

In [104]:
window_length = 1

In [105]:
actor = Sequential()
# The network's input fits the observation space of the env
actor.add(Flatten(input_shape=(window_length,) + (4,)))  # observation_space.shape != (4,).. ## todo: correct it
actor.add(Dense(16, activation='relu'))
actor.add(Dense(17, activation='relu'))
# The network output fits the action space of the env
actor.add(Dense(nb_actions,
                kernel_initializer=initializers.RandomNormal(stddev=1e-5),
                activation='sigmoid',
                kernel_regularizer=regularizers.l2(1e-2)))
actor.add(tf.keras.layers.Lambda(lambda x: x * 100))
print(actor.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 17)                289       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 36        
_________________________________________________________________
lambda (Lambda)              (None, 2)                 0         
Total params: 405
Trainable params: 405
Non-trainable params: 0
_________________________________________________________________
None


In [106]:
#todo : Normalise inputs(action and oservation)
action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(window_length,) + (4,), name='observation_input')
# (using keras functional API)
flattened_observation = Flatten()(observation_input)
x = Concatenate()([action_input, flattened_observation])
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
#x = Dense(32, activation='relu')(x)
x = Dense(1, activation='linear')(x)
critic = Model(inputs=(action_input, observation_input), outputs=x)
print(critic.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observation_input (InputLayer)  [(None, 1, 4)]       0                                            
__________________________________________________________________________________________________
action_input (InputLayer)       [(None, 2)]          0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 4)            0           observation_input[0][0]          
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 6)            0           action_input[0][0]               
                                                                 flatten_1[0][0]              

In [107]:
memory = SequentialMemory(
    limit=5000,
    window_length=1
)

In [108]:
# Create a random process for exploration during training
# this is essential for the DDPG algorithm
random_process = OrnsteinUhlenbeckProcess(
    theta=0.5,
    mu=0.0,
    sigma=0.1,
    dt=0.001,
    sigma_min=0.05,
    n_steps_annealing=85000,
    size=2
)

In [109]:
agent = DDPGAgent(
    # Pass the previously defined characteristics
    nb_actions=nb_actions,
    actor=actor,
    critic=critic,
    critic_action_input=action_input,
    memory=memory,
    random_process=random_process,

    # Define the overall training parameters
    nb_steps_warmup_actor=2048,
    nb_steps_warmup_critic=1024,
    target_model_update=1000,
    gamma=0.99,
    batch_size=128,
    memory_interval=2
)

In [110]:
agent.compile([Adam(lr=3e-5), Adam(lr=3e-3)])

In [ ]:
# Training
agent.fit(
    env_h,
    nb_steps=10000,
    nb_max_start_steps=0,
    nb_max_episode_steps=500,
    visualize=False,
    action_repetition=1,
    verbose=2,
    log_interval=50,

)

Training for 10000 steps ...
  500/10000: episode: 1, duration: 0.344s, episode steps: 500, steps per second: 1454, episode reward: 4470.219, mean reward:  8.940 [ 0.011, 16.211], mean action: 50.012 [49.918, 50.119],  loss: --, mean_q: --
 1000/10000: episode: 2, duration: 0.282s, episode steps: 500, steps per second: 1773, episode reward: 4467.662, mean reward:  8.935 [ 0.004, 16.207], mean action: 49.895 [49.752, 50.001],  loss: --, mean_q: --
 1500/10000: episode: 3, duration: 2.463s, episode steps: 500, steps per second: 203, episode reward: 4479.938, mean reward:  8.960 [ 0.011, 16.250], mean action: 50.110 [50.045, 50.212],  loss: 1.726865, mean_q: -0.010476
 2000/10000: episode: 4, duration: 2.309s, episode steps: 500, steps per second: 217, episode reward: 4480.573, mean reward:  8.961 [ 0.022, 16.250], mean action: 50.148 [50.075, 50.217],  loss: 0.721814, mean_q: 0.100356
 2500/10000: episode: 5, duration: 2.569s, episode steps: 500, steps per second: 195, episode reward: 29

In [ ]:
# Test the agent
hist = agent.test(
    env_h,
    nb_episodes=1,
    action_repetition=5,
    visualize=False,
    nb_max_episode_steps=2000
)